In [10]:
from collections import deque
import heapq

def process_large_file(file_path, output_path):
    """Processes a file efficiently to maximize score and adhere to pairing rules."""
    portrait_images = []
    landscape_images = deque()
    final_frames = []
    total_score = 0

    def parse_line(line):
        """Parse each line into a dictionary."""
        parts = line.strip().split()
        orientation = parts[0]
        tags = set(parts[2:])
        return {"orientation": orientation, "tags": tags}

    def calculate_pair_score(tags1, tags2):
        """Calculate score for a pair of images."""
        # Consider optimizing set operations for large tag sets if necessary
        common_tags = len(tags1 & tags2)
        tags_in_1_not_in_2 = len(tags1 - tags2)
        tags_in_2_not_in_1 = len(tags2 - tags1)
        return min(common_tags, tags_in_1_not_in_2, tags_in_2_not_in_1)

    # Parse input file
    with open(file_path, 'r') as file:
        lines = file.readlines()

        for idx, line in enumerate(lines[1:]):  # Skip the first line (batch count)
            image = parse_line(line)
            image["id"] = [idx]
            if image["orientation"] == "P":
                portrait_images.append(image)
            elif image["orientation"] == "L":
                landscape_images.append(image)

    # Use a priority queue to pair "P" images optimally
    pq = []
    used_indices = set()
    for i in range(len(portrait_images)):
        for j in range(i + 1, len(portrait_images)):
            if i not in used_indices and j not in used_indices:
                score = calculate_pair_score(portrait_images[i]["tags"], portrait_images[j]["tags"])
                heapq.heappush(pq, (-score, i, j))

    while pq and len(used_indices) < len(portrait_images):
        score, i, j = heapq.heappop(pq)
        if i not in used_indices and j not in used_indices:
            used_indices.add(i)
            used_indices.add(j)
            combined_tags = portrait_images[i]["tags"] | portrait_images[j]["tags"]
            total_score += -score
            final_frames.append({
                "id": portrait_images[i]["id"] + portrait_images[j]["id"],
                "orientation": "P",
                "tags": combined_tags,
            })

    # Add standalone "L" images as frames
    while landscape_images:
        img = landscape_images.popleft()
        final_frames.append({
            "id": img["id"],
            "orientation": "L",
            "tags": img["tags"],
        })

    # Calculate total number of frames
    frame_count = len(final_frames)

    # Write output to file
    with open(output_path, 'w') as out_file:
        out_file.write(f"Total Score: {total_score}\n")
        out_file.write(f"Number of Frames: {frame_count}\n")
        for frame in final_frames:
            frame_id = " ".join(map(str, frame["id"]))
            out_file.write(f"{frame_id}\n")

    print(f"Processing completed. Total Score: {total_score}, Number of Frames: {frame_count}")

# File paths
input_file = "./Data/10_computable_moments.txt"  # Adjust file path as needed
output_file = "output_frames.txt"

# Process the file
process_large_file(input_file, output_file)

Processing completed. Total Score: 426, Number of Frames: 750


In [ ]:
def calculate_local_satisfaction(f1, f2):
    """Calculate the Local Robotic Satisfaction between two frameglasses."""
    common = len(f1 & f2)
    f1_diff = len(f1 - f2)
    f2_diff = len(f2 - f1)
    return min(common, f1_diff, f2_diff)

def greedy_ordering(frameglasses):
    """Order frameglasses greedily to maximize Global Robotic Satisfaction."""
    ordered = [frameglasses.pop(0)]  # Start with the first frameglass
    total_score = 0

    while frameglasses:
        best_next = None
        best_score = -1

        for f in frameglasses:
            score = calculate_local_satisfaction(ordered[-1], f)
            if score > best_score:
                best_score = score
                best_next = f

        ordered.append(best_next)
        total_score += best_score
        frameglasses.remove(best_next)

    return ordered, total_score

def process_paintings(paintings):
    """Convert paintings into frameglasses."""
    landscapes = []
    portraits = []

    for painting in paintings:
        if painting[0] == 'L':
            landscapes.append(set(painting[2:]))
        elif painting[0] == 'P':
            portraits.append(set(painting[2:]))

    # Pair Portraits greedily
    portrait_frameglasses = []
    while len(portraits) > 1:
        best_pair = None
        best_diversity = -1

        # Find the best pair to maximize tag diversity
        for i in range(len(portraits)):
            for j in range(i + 1, len(portraits)):
                diversity = len(portraits[i] | portraits[j])  # Union of tags
                if diversity > best_diversity:
                    best_diversity = diversity
                    best_pair = (i, j)

        # Combine the best pair into a frameglass
        i, j = best_pair
        combined_frameglass = portraits[i] | portraits[j]
        portrait_frameglasses.append(combined_frameglass)

        # Remove paired portraits
        portraits.pop(j)  # Pop larger index first
        portraits.pop(i)

    # Add any leftover single portrait as a frameglass
    if portraits:
        portrait_frameglasses.append(portraits.pop())

    landscape_frameglasses = landscapes

    return portrait_frameglasses + landscape_frameglasses

# Example Input
paintings = [
    ['L', 3, 'animals', 'fear', 'war'],
    ['P', 2, 'smile', 'woman'],
    ['P', 2, 'woman', 'pearl'],
    ['L', 3, 'fear', 'raft', 'survivors']
]

# Process paintings into frameglasses
frameglasses = process_paintings(paintings)

# Order frameglasses greedily
optimal_order, max_score = greedy_ordering(frameglasses)

# Output the results
print("Optimal Order:", optimal_order)
print("Max Global Satisfaction:", max_score)


In [13]:
import os

def calculate_local_satisfaction(f1, f2):
    """Calculate the Local Robotic Satisfaction between two frameglasses."""
    common = len(f1 & f2)
    f1_diff = len(f1 - f2)
    f2_diff = len(f2 - f1)
    return min(common, f1_diff, f2_diff)

def greedy_ordering(frameglasses):
    """Order frameglasses greedily to maximize Global Robotic Satisfaction."""
    ordered = [frameglasses.pop(0)]  # Start with the first frameglass
    order_indices = [[0]]  # Keep track of indices for the output file
    total_score = 0

    while frameglasses:
        best_next = None
        best_score = -1
        best_index = -1

        for idx, f in enumerate(frameglasses):
            score = calculate_local_satisfaction(ordered[-1][1], f[1])
            if score > best_score:
                best_score = score
                best_next = f
                best_index = idx

        ordered.append(best_next)
        order_indices.append(best_next[0])
        total_score += best_score
        frameglasses.pop(best_index)

    return ordered, total_score

def process_paintings(file_path):
    """Read and process paintings from the input file."""
    with open(file_path, 'r') as f:
        lines = f.readlines()

    paintings = []
    for line in lines[1:]:
        parts = line.strip().split()
        paintings.append([parts[0], int(parts[1]), *parts[2:]])

    landscapes = []
    portraits = []

    for idx, painting in enumerate(paintings):
        if painting[0] == 'L':
            landscapes.append(([idx], set(painting[2:])))
        elif painting[0] == 'P':
            portraits.append(([idx], set(painting[2:])))

    # Pair Portraits greedily
    portrait_frameglasses = []
    while len(portraits) > 1:
        best_pair = None
        best_diversity = -1

        # Find the best pair to maximize tag diversity
        for i in range(len(portraits)):
            for j in range(i + 1, len(portraits)):
                diversity = len(portraits[i][1] | portraits[j][1])  # Union of tags
                if diversity > best_diversity:
                    best_diversity = diversity
                    best_pair = (i, j)

        # Combine the best pair into a frameglass
        i, j = best_pair
        combined_indices = portraits[i][0] + portraits[j][0]
        combined_frameglass = portraits[i][1] | portraits[j][1]
        portrait_frameglasses.append((combined_indices, combined_frameglass))

        # Remove paired portraits
        portraits.pop(j)  # Pop larger index first
        portraits.pop(i)

    # Add any leftover single portrait as a frameglass
    if portraits:
        portrait_frameglasses.append(portraits.pop())

    landscape_frameglasses = landscapes

    return portrait_frameglasses + landscape_frameglasses

def process_and_output(file_path, output_file_path):
    """Process input and write the output to a file."""
    frameglasses = process_paintings(file_path)
    ordered_frameglasses, max_score = greedy_ordering(frameglasses)
    
    with open(output_file_path, 'w') as f:
        for frameglass in ordered_frameglasses:
            f.write(' '.join(map(str, frameglass[0])) + '\n')  # Write indices of the paintings
        f.write(str(max_score) + '\n')  # Write the total score on the last line

# Main Function
def main():
    # Define input and output file paths
    input_files = [
     #   "0_example.txt",
       "./Data/10_computable_moments.txt",
       # "11_randomizing_paintings.txt",
      #  "110_oily_portraits.txt"
    ]
    output_dir = "output_files"
    os.makedirs(output_dir, exist_ok=True)

    for input_file in input_files:
        output_file = os.path.join(output_dir, os.path.basename(input_file).replace(".txt", "_output.txt"))
        process_and_output(input_file, output_file)
        print(f"Processed {input_file}, output saved to {output_file}")

if __name__ == "__main__":
    main()


Processed ./Data/10_computable_moments.txt, output saved to output_files\10_computable_moments_output.txt
